In [4]:

import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt 
import time
from src.utils.cp_utils import * 
from src.models.conformal_model import ConformalModel, ConformalModelLogits
import torch.backends.cudnn as cudnn
import random
from src.data.make_cifar10_dataset import CIFAR10, get_img_transformer
from src.models.cifar10_conv_model import Cifar10ConvModel



In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
train_dataset = CIFAR10(split="train", root='../data/processed', download=True,
                      transform=get_img_transformer())
test_dataset = CIFAR10(split="test", root='../data/processed', download=True,
                      transform=get_img_transformer())
test_dataloader = torch.utils.data.DataLoader(
        test_dataset, batch_size=32)
calib_dataset = CIFAR10(split="calib", root='../data/processed', download=True,
                      transform=get_img_transformer())
calib_dataloader = torch.utils.data.DataLoader(
        calib_dataset, batch_size=32)
print(train_dataset.data.shape)
print(test_dataset.data.shape)
print(calib_dataset.data.shape)
n_classes = len(set(test_dataset.targets))
classes = {index: name for name, index in test_dataset.class_to_idx.items()}


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
(40000, 32, 32, 3)
(10000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
model = Cifar10ConvModel().cpu()
model.load_state_dict(torch.load("../models/trained_model.pt"))

In [13]:
model = ConformalModel(model=premodel,calib_loader=calib_dataloader, alpha=0.05)

Begin Platt scaling.
Computing logits for model (only happens once).


100%|██████████| 313/313 [00:27<00:00, 11.46it/s]


Optimal T=1.252213954925537
[0 0 0 ... 0 0 0]
(tensor([12.5500, -2.6525,  0.2643, -1.6349, -1.1045, -5.4705, -2.4388, -1.7302,
         2.9399, -0.7230]), tensor(0))
paramtune_logits[0]
(tensor([-2.0604,  2.0145, -2.1122, -0.3227, -3.2370,  2.2179, -2.5628, -1.2019,
         0.1952,  7.0692]), tensor(9))


In [14]:
validate(test_dataloader, model, print_bool=True)

N: 10000 | Time: 0.039 (0.075) | Cvg@1: 0.812 (0.877) | Cvg@5: 0.938 (0.991) | Cvg@RAPS: 0.875 (0.933) | Size@RAPS: 1.250 (1.293)


(0.877, 0.9912, 0.9331, 1.2928)

## Load pretrained

In [20]:
premodel = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20",  pretrained=True)

Using cache found in /Users/abdulrahman/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [18]:
model = torch.hub.download_url_to_file('https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet20-4118986f.pt', '../models/cifar10_resnet20.pt')

  0%|          | 0.00/1.09M [00:00<?, ?B/s]

In [19]:
model.eval()

AttributeError: 'NoneType' object has no attribute 'eval'